In [203]:
# Import List
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import numpy as np

import pandas as pd


# 实验2

## 标准BP算法

首先需要对数据进行归一化处理，便于神经网络的使用。

In [204]:
# 读取所有数据（不包括第一行参数）
wine_data = np.genfromtxt('wine_data.csv', delimiter=',', skip_header=1)

# 读取所有特征数据，排除最后一列（列标签）
X = wine_data[:, 0:13]
# 标签列读取
y = wine_data[:, 13]

# 划分70%训练集，30%测试集
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size=0.7)
# 数据标准化
label_train = LabelBinarizer().fit_transform(y_train)

设置神经网络参数。包括学习率、迭代次数、神经节点设置。

In [205]:
# 学习率
rate = 0.11
# 迭代次数
step = 10000

# 随机设置神经权值，13输入，3输出中间隐藏层100个神经元
v = np.random.random((13, 100)) * 2 - 1  # 输入层->隐层
w = np.random.random((100, 3)) * 2 - 1  # 隐层->输出层

激活函数$sigmoid()$与其导数函数如下。

In [206]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def de_sigmoid(x):
    return x * (1 - x)

创建训练函数
标准BP算法核心要点如下：

1. 随机初始化节点的权值。
2. 按照激活函数公式计算出样本误差。
3. 根据样本误差反推出权值梯度项。
4. 更新节点权值。

反复执行上述过程，直到达到设定的训练次数。

In [207]:
def standard_bp_train(train_x, train_y, train_rate, train_step):
    # 导入神经节点
    global v, w
    for n in range(train_step + 1):
        # 随机选择样本
        i = np.random.randint(train_x.shape[0])
        x = train_x[i]
        x = np.atleast_2d(x)

        # 节点激活
        L1 = sigmoid(np.dot(x, v))  # 输入层->隐层
        L2 = sigmoid(np.dot(L1, w))  # 隐层->输出层

        # 误差反向反馈
        L2_delta = (train_y[i] - L2) * de_sigmoid(L2)  # 输出层->隐层
        L1_delta = L2_delta.dot(w.T) * de_sigmoid(L1)  # 隐层->输入层

        # 权值更新
        w = w + train_rate * L1.T.dot(L2_delta)  # 隐层->输出层
        v = v + train_rate * x.T.dot(L1_delta)  # 输入层->隐层

        # 每1000次训练对网络进行测试，输出测试准确率
        if n % 1000 == 0:
            output = predict(x_test)
            predictions = np.argmax(output, axis=1)  # 选择概率最大的输出点作为结果，输出所在列编号，对应分类结果
            acc = np.mean(np.equal(predictions, y_test))  # 对比测试样本，求准确率
            print("step:", n, "acc:", acc)


def predict(x):
    L1 = sigmoid(np.dot(x, v))
    L2 = sigmoid(np.dot(L1, w))
    return L2


standard_bp_train(x_train, label_train, rate, step)

/Users/orekimai/.conda/envs/MachineLearning/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


step: 0 acc: 0.37037037037037035
step: 1000 acc: 0.37037037037037035
step: 2000 acc: 0.37037037037037035
step: 3000 acc: 0.25925925925925924
step: 4000 acc: 0.37037037037037035
step: 5000 acc: 0.37037037037037035
step: 6000 acc: 0.37037037037037035
step: 7000 acc: 0.37037037037037035
step: 8000 acc: 0.25925925925925924
step: 9000 acc: 0.37037037037037035
step: 10000 acc: 0.37037037037037035


通过运行标准BP算法可以发现，单层神经网络中神经点之间的权重在每次放入样本训练后都会发生改变，改变次数与训练次数相关，即：标准BP算法更新规则是基于每次样本而去修改。

## 累计BP算法
累计BP算法

In [208]:
def sum_bp_train(train_x, train_y, train_rate, train_step):
    return



## 构建神经网络

使用TensorFlow搭建神经网络。

In [209]:
# Import List
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, datasets, Sequential

创建TensorFlow神经网络，设置输入层、隐藏层和输出层的大小。

导入数据，并划分训练集与测试集。

In [210]:
# 读取所有数据（不包括第一行参数）
wine_data_tf = np.genfromtxt('wine_data.csv', delimiter=',', skip_header=1)

# 读取所有特征数据，排除最后一列（列标签）
X_tf = wine_data_tf[:, 0:13]
# 标签列读取
y_tf = wine_data_tf[:, 13]

# 划分70%训练集，30%测试集
x_train_tf, x_test_tf, y_train_tf, y_test_tf = train_test_split(X_tf, y_tf, test_size=0.3, train_size=0.7)
# 数据标准化
label_train_tf = LabelBinarizer().fit_transform(y_train_tf)

In [211]:
# 使用keras的层堆叠创建神经网络，包含输入层、隐层和输出层
model = tf.keras.Sequential([
    # layers.Input(shape=x_train_tf.shape[1:]),
    layers.Dense(16, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# 参数设置
LEARNING_RATE = 0.001

# 通过compile调整网络学习参数
model.compile(
    # 优化器
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    # 损失函数
    loss='sparse_categorical_crossentropy',
    # 评估函数
    metrics=['sparse_categorical_accuracy'])



向模型输入数据进行训练

In [ ]:
model.fit(x_train_tf, label_train_tf,
          batch_size=100,
          epochs=5,
          validation_data=(x_test_tf, y_test_tf),
          validation_freq=1)

model.summary()